In [ ]:
import warnings
import socket
from time import sleep
import math
from time import time
from guizero import App, Text, TextBox, PushButton, Slider, Picture
import cv2
import os
import matplotlib.pyplot as plt
##import tensorflow as tf
##print(tf.__version__)
from deepface import DeepFace
import tkinter as tk
from tkinter import filedialog
import serial
import threading
from secret import IP



ser = serial.Serial('/dev/cu.usbmodem143101', baudrate = 115200, timeout = 1)

#! important: make a file called secret in the same directory and write your IP as a string.
#! if you want to run the Pure Data and Python in the same machine, you may also write 'localhost' 
#! e.g. IP = "192.168.1.1" or IP = "localhost"
#from secret import IP

#IP = '192.168.86.64'
port = 7000



class PDSocket:
    """PD-PI Socket"""

    def __init__(self, sock=None):
        if sock is None:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        else:
            self.sock = sock

    def connect(self, host, port):
        self.sock.connect((host, port))

    def close(self):
        self.sock.close()

    def send(self, msg):
        pdmsg = ';'.join([f'{k} {v}' for k, v in msg.items()])
        sent = self.sock.sendall(bytes(pdmsg + ';', 'ascii'))
        if sent == 0:
            raise RuntimeError("socket connection broken")

    def receive(self):
        msg = str(self.sock.recv(1024), 'ascii')
        if msg == b'':
            raise RuntimeError("socket connection broken")
        return msg[:-2].split(';\n')


def main(host=IP, port=7000):
    
    times = []




            
    
    print(f"> Connecting to '{host}' at port '{port}'")
    socket = PDSocket()
    socket.connect(host, port)
    

    # ?inputs 
    start = 1  # switch
    emotion = 1  # mood
    dis = 0  # dissonance
    dspToggle = 0  # dsp

    #! killswitch
    #start = 0
    #dsp = 0
        
    def update_face():
        mood = 1
        dis = 0
        valid = True
        while (valid):
            root = tk.Tk()
            root.withdraw()
            file_path = filedialog.askopenfilename()
            print(file_path)
            try:
                cures = cv2.imread(file_path)
                plt.imshow(cures)
                predictions = DeepFace.analyze(cures, actions = ['emotion'])
                print(predictions)
                print(type(predictions))
                if predictions.get('dominant_emotion') == 'anger' or predictions.get('dominant_emotion') == 'disgust':
                    dis = 1
                    mood = 5
                elif predictions.get('dominant_emotion') == 'surprise':
                    dis = 1
                    mood = 1
                elif predictions.get('dominant_emotion') == 'fear':
                    dis = 1
                    mood = 2

                elif predictions.get('dominant_emotion') == 'neutral':
                    dis = 0
                    mood = 3
                elif predictions.get('dominant_emotion') == 'happy':
                    dis = 0
                    if predictions.get('emotion').get('neutral') > 30:
                        mood = 2
                    else:
                        mood = 1
                elif predictions.get('dominant_emotion') == 'sad':
                    dis = 0
                    if predictions.get('emotion').get('neutral') > 30:
                        mood = 4
                    else:
                        mood = 5

                print("mood value is: "+ str(mood))
                print("dis: "+ str(dis))
                print('sucessfully ran')

                valid = False
            except ValueError:
                print('no face was found. try again')
            except TypeError:
                print('bad file type. only accept png or jpg')
        socket.send({'emotion': mood, 'dis': dis})

    socket.send({'tempo': 100})
    
    values = {'start': start, 'emotion': emotion, 'dis': dis, 'dspToggle': dspToggle}
    
    try:
        print("init values")
        socket.send(values)

        def arduinoCom():   
            while True:
                print("ahhhhh")
                arduinoData = ser.readline().decode('ascii').replace(" ", "")

                if arduinoData == '':
                    print('empty')
                else:
                    print(arduinoData)
                    socket.send({'tempo': int(arduinoData)})

        
        t1 = threading.Thread(target=arduinoCom, name='t1')
        t1.start()
        app = App(title="Yousic Home")
        welcome_message = Text(app, text="Mood Generation Input", size=40, font="Times New Roman", color="lightblue")
        update_face = PushButton(app, command=update_face, text="Upload image of your face")
        my_name = TextBox(app, width=50)
        app.display()
        
        
                    
                    
    except KeyboardInterrupt:
        print("> Closing socket")
        socket.close()


if __name__ == '__main__':
    main()


> Connecting to '192.168.86.64' at port '7000'
init values
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
/Users/paxysnowman/Desktop/Screen Shot 2021-07-23 at 09.56.42.png
empty
ahhhhh
empty
ahhhhh
{'region': {'x': 180, 'y': 67, 'w': 429, 'h': 576}, 'emotion': {'angry': 0.08201049547518763, 'disgust': 6.551556129396054e-10, 'fear': 0.1297090861959211, 'happy': 0.0001717298789612772, 'sad': 0.503359997845292, 'surprise': 0.0014882782102502467, 'neutral': 99.28326006431938}, 'dominant_emotion': 'neutral'}
<class 'dict'>
mood value is: 3
dis: 0
sucessfully ran
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
empty
ahhhhh
/Users/paxysnowman/Desktop/Screen Shot 2021-07-27 at 10.22.50.png
empty
ahhhhh
empty
ahhhhh
{'region': {'x': 219, 'y': 118, 'w': 179, 'h': 253}, 'emoti